# Wstęp do uczenia maszynowego. Projekt nr 1.

*Maciej Borkowski, Michał Chęć
21.04.2023r.*

Walidowali: Alicja Charuza, Mateusz Gałęziewski

Zadaniem projektowym jest zrealizowanie zadania klasyfikacji binarnej na zbiorze danych numerycznych ze strony
[https://www.kaggle.com/datasets/nextbigwhat/dataset-1](https://www.kaggle.com/datasets/nextbigwhat/dataset-1)


# UWAGA po pierwsze: brak pliku 'requirements', który jest niezbędny do pełnej reprodukcji wyników projektu


In [33]:
# ładujemy potrzebne pakiety
import pandas as pd
import numpy as np
from tabulate import tabulate
import matplotlib.pyplot as plt

from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_selector

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, accuracy_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.feature_selection import SelectKBest, SelectFromModel, SequentialFeatureSelector, RFE, VarianceThreshold
from sklearn.decomposition import PCA

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.ensemble import AdaBoostClassifier

import warnings
import os
import sys
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    os.environ["PYTHONWARNINGS"] = "ignore"

np.random.seed(42)

# 1. Import i podział danych

In [13]:
#wczytujemy całą ramkę danych
dataset = pd.read_csv("dataset_1.csv")
df = dataset.copy()

In [14]:
#dzielimy dane, tworzymy zbiór do ewaluacji i do testów
train_set, eval_set = train_test_split(df, test_size=0.3, random_state=42)
train_df, test_df = train_test_split(train_set, test_size=0.3, random_state=42)
df = train_df.copy()

In [15]:
#czy zmienna objaśniana ma mniej więcej ten sam współczynnik 1 w obu zbiorach
[dataset[dataset.target == 1].size/dataset.size,
 eval_set[eval_set.target == 1].size/eval_set.size,
 train_set[train_set.target == 1].size/train_set.size]

[0.03982, 0.04006666666666667, 0.039714285714285716]

# 2. EDA i preprocessing

Na początek parę słów odnośnie eksploracyjnej analizy danych i preprocessingu. 
Zespół budowy poprawnie wykonuje wstępną analizę. Sprawdza występowanie braków danych, typów danych i ich rozmiar, czy zbiory: treningowy, testowy, walidacyjny są zbalansowane.
Dobrym pomysłem zespołu budowy również jest oddzielna analiza eksploracyjna kolumn zawierających wartości w postaci liczb całkowitych jak i tych zawierających liczby zmiennoprzecinkowe. 
Pozbywając się kolumn zależnych, stałych, prawiestałych i zduplikowach zmniejszyli wymiarowość danych, co również uznajemy za słuszne.

Dziwnym rozwiązaniem natomiast jest stosowanie oddzielnych preprocessingów dla każdego modelu z osobna. Biorąc jednakże pod uwagę syntetyczność danych oraz świadomość zespołu budowy jesteśmy w stanie zrozumieć takie podejście.  Na plus jest także tworzenie przez zespół budowy dydykowanych narzęci do eksploracyjnej analizy danych.

Zespół budowy stworzył również od podstaw klasę 'ColumnRemover' do redukcji ilości kolumn, funkcja jest skomplikowana co ciekawe przyjmuje nawet parametry 


In [16]:
df.shape

(24500, 301)

In [17]:
def rem_by_Var_threshold(df, threshold):
    X = df.drop('target', axis=1)
    
    selector = VarianceThreshold(threshold=threshold)
    selector.fit(X)
    selected_features = X.columns[selector.get_support()]
    X_selected = X[selected_features]
    return X_selected.join(df.target)

### Poniżej widzimy ciekawy pomysł budujących
Jak się później niestety okaże to przez jedną z poniższch funkcji nie będzie można przeprowadzić votingu.

In [18]:
class ColumnRemover(BaseEstimator, TransformerMixin):

    def __init__(self, threshold_constant, threshold_corr, n_info_vals):
        self.threshold_constant = threshold_constant
        self.threshold_corr = threshold_corr
        self.n_info_vals = n_info_vals
        self.columns_to_remove = []
        self.columns_to_keep = []

    def fit(self, X, y=None):
        # usuwanie zduplikowanych kolumn
        self.columns_to_remove.extend(X.loc[:, X.T.duplicated()].columns.tolist())

        # usuwanie stałych i prawie stałych kolumn
        for column in X.columns:
            if X[column].value_counts(normalize=True).iloc[0] >= self.threshold_constant:
                self.columns_to_remove.append(column)

        # usuwanie skorelowanych kolumn, jeśli pierwsza kolumna jest ciągła stosujemy korelację pearsona,
        # jeśli dyskretna to korelację spearmana; pierwsza kolumna będzie typu takiego samego jak cały X ponieważ
        # klasę tą stosujemy w column tranformerze ograniczając podzbiór kolumn do okreslonego typu
        if X.dtypes[0] == 'float64':
            corr = X.corr(method='pearson')
        else:
            corr = X.corr(method='spearman')
        corr = corr[corr > self.threshold_corr]
        dependent_columns = corr.apply(lambda row: row[row > 0].index, axis=1)
        for j in range (len(dependent_columns)):
            for k in dependent_columns[j]:
                if k is not dependent_columns.index[j]:
                    if k not in dependent_columns.index[0:j]:
                        self.columns_to_remove.append(k)

        # usuwanie kolumn nie niosących informacji
        amount_of_ones = y[y == 1].shape[0]
        X = X.join(y)
        for column in X.columns:
            tmp = X.groupby(column)['target'].agg(['sum','count']).sort_values('sum',ascending = False).reset_index()
            if any(tmp[column] == 0) and (tmp.loc[tmp[column] == 0, 'sum'] > amount_of_ones - self.n_info_vals).bool():
                self.columns_to_remove.append(column)
        X.drop('target', axis=1, inplace=True)

        self.columns_to_keep = [col for col in X.columns if col not in self.columns_to_remove]

        return self

    def transform(self, X):
        return X[self.columns_to_keep]

Zespół budowy stworzył funkcję 'show_scores' do wyświetlania metryk. Warto wiedzieć, że istnieje gotowa i importowana z 'sklearn.metrics' funkcja 'classification_report' służąca do tego samego.
Minusem poniższej funkcji jest chociażby to, że nie każdy model przyjmuje metodę 'predict_proba'.
Jeśli budujący używaliby takiego modelu jak np. MLPClassifier musieliby napisać drugą funkcję używającą metodę 'decision_function'.

In [19]:
def show_scores(clf, X, y):
    y_pred = clf.predict(X)
    y_pred_prob = clf.predict_proba(X)
    print(tabulate(confusion_matrix(y, y_pred), headers=['Predicted 0', 'Predicted 1'], tablefmt='orgtbl'))
    print()
    print(f'accuracy:              {round(accuracy_score(y, y_pred), 4)}')
    print(f'precision:             {round(precision_score(y, y_pred), 4)}')
    print(f'recall:                {round(recall_score(y, y_pred), 4)}')
    print(f'f1:                    {round(f1_score(y, y_pred), 4)}')
    print(f'roc_auc_discrete:      {round(roc_auc_score(y, y_pred), 4)}')
    print(f'roc_auc_continuous:    {round(roc_auc_score(y, y_pred_prob[:, 1]), 4)}')

Przejdźmy zatem do modeli

# 3. Regresja logistyczna

## 3.1 Preprocessing

In [72]:
#Kod zepołu budowy
# zarówno dla zmiennych dyskretnych i ciągłych zaspół budowy stosuje transformator ColumnRemover z różnymi parametrami 
# w kolejnych krokach będzie szukać najlepszej ich kombinacji

#hot encoding zmiennych dyskretnych - jako kategoryczne
int_transformer = Pipeline([
    ('int', ColumnRemover(0.9995, 0.99, 1)),
    ('one_hot', OneHotEncoder(handle_unknown='ignore', sparse_output=False, dtype='int64'))])

#Zespół budowy upiera się że standaryzacja jest lepsza od normalizacji min-max dla tego modelu
float_transformer = Pipeline([
    ('float', ColumnRemover(0.9999, 0.99, 10)),
    ('standard_scaler', StandardScaler())])

col_transformer = ColumnTransformer([
    ('int_pipe', int_transformer, make_column_selector(dtype_include=np.int64)),
    ('float_pipe', float_transformer, make_column_selector(dtype_include=np.float64))
])

## 3.2 Trening pierwszego modelu

In [73]:
X_train = train_df.drop('target', axis=1)
y_train = train_df.target
x_eval = eval_set.drop('target', axis=1)
y_eval = eval_set.target

In [15]:
clf = Pipeline([
    ('preprocessing', col_transformer),
    ('model', LogisticRegression(random_state=42))])

clf.fit(X_train, y_train)

# wyniki dla danych treningowych
show_scores(clf, X_train, y_train)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         23550 |            10 |
|           934 |             6 |

accuracy:              0.9615
precision:             0.375
recall:                0.0064
f1:                    0.0126
roc_auc_discrete:      0.503
roc_auc_continuous:    0.8057


In [16]:
# wyniki dla danych walidacyjnych 
show_scores(clf, x_eval, y_eval)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         14391 |             8 |
|           600 |             1 |

accuracy:              0.9595
precision:             0.1111
recall:                0.0017
f1:                    0.0033
roc_auc_discrete:      0.5006
roc_auc_continuous:    0.7864


### Prównanie ze zbiorem testowym:
accuracy:              0.9556

precision:             0.0556

recall:                0.0022

f1:                    0.0043

roc_auc_discrete:      0.5003

roc_auc_continuous:    0.7813

In [17]:
clf = Pipeline([
    ('preprocessing', col_transformer),
    ('model', LogisticRegression(random_state=42, class_weight='balanced'))])

clf.fit(X_train, y_train)
show_scores(clf, X_train, y_train)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         16261 |          7299 |
|           207 |           733 |

accuracy:              0.6936
precision:             0.0913
recall:                0.7798
f1:                    0.1634
roc_auc_discrete:      0.735
roc_auc_continuous:    0.8125


In [18]:
#wyniki dla danych walidacyjnych
show_scores(clf, x_eval, y_eval)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|          9873 |          4526 |
|           163 |           438 |

accuracy:              0.6874
precision:             0.0882
recall:                0.7288
f1:                    0.1574
roc_auc_discrete:      0.7072
roc_auc_continuous:    0.7848


### Prównanie ze zbiorem testowym:
accuracy:              0.683

precision:             0.0932

recall:                0.7333

f1:                    0.1655

roc_auc_discrete:      0.707

roc_auc_continuous:    0.7784

### Minimalnie lepiej...

## 3.3 Strojenie hiperparametrów i dobór cech

In [19]:
X_train = train_df.drop('target', axis=1)
y_train = train_df.target
x_eval = eval_set.drop('target', axis=1)
y_eval = eval_set.target

### 3.3.2 Regularyzacja modelu

Zespół budowy pragnie zapobiec overfittingowi. 

Zajmuje się regularyzacją modelu regresji liniowej - będzie sprawdzać odwrotność współczynnika regularyzacji oraz rodzaj kary (l1, l2). Zastosują parametry ColumnRemovera ustalone w poprzednich podpunktach.


In [20]:
#Kod zespołu budowy
#parametry ColumnRemovera na znalezione w 3.3.1
int_transformer = Pipeline([
    ('int', ColumnRemover(0.9998, 1, 0)),
    ('one_hot', OneHotEncoder(handle_unknown='ignore', sparse_output=False, dtype='int64'))])

float_transformer = Pipeline([
    ('float', ColumnRemover(0.9996, 0.97, 0)),
    ('standard_scaler', StandardScaler())])

col_transformer = ColumnTransformer([
    ('int_pipe', int_transformer, make_column_selector(dtype_include=np.int64)),
    ('float_pipe', float_transformer, make_column_selector(dtype_include=np.float64))
])

X_train = col_transformer.fit_transform(train_df.drop('target', axis=1), train_df.target)
y_train = train_df.target
x_eval = col_transformer.transform(eval_set.drop('target', axis=1))
y_eval = eval_set.target

Sprawdzamy najlepsze parametry i wyniki dla najlepszego modelu

In [46]:
#Wyniki 
#reg_search.best_score_ = 0.7879, reg_search.best_params_ = {'C': 0.11288378916846883, 'penalty': 'l1'}

0.7879 {'C': 0.11288378916846883, 'penalty': 'l1'}


In [21]:
#dla danych treningowych
clf = LogisticRegression(random_state=42, class_weight='balanced', solver='liblinear', C=0.11288378916846883, penalty='l1')
clf.fit(X_train, y_train)
show_scores(clf, X_train, y_train)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         16357 |          7203 |
|           213 |           727 |

accuracy:              0.6973
precision:             0.0917
recall:                0.7734
f1:                    0.1639
roc_auc_discrete:      0.7338
roc_auc_continuous:    0.8121


In [23]:
#dla danych walidacyjnych 
show_scores(clf, x_eval, y_eval)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|          9918 |          4481 |
|           169 |           432 |

accuracy:              0.69
precision:             0.0879
recall:                0.7188
f1:                    0.1567
roc_auc_discrete:      0.7038
roc_auc_continuous:    0.7893


### Prównanie ze zbiorem testowym:
accuracy:              0.6879

precision:             0.0944

recall:                0.7311

f1:                    0.1672

roc_auc_discrete:      0.7085

roc_auc_continuous:    0.782

### 3.3.3 Dobór zmiennych nie wymagajacy modelu

Korelację uwzględnili w transformatorze ColumnRemover. Technika - SelectKBest. Najlepsze K zostało wybrane podczas Grid Searchu przez zespół budowy. Poniżej jest ostatecznie wybrana wartość dla najlepszego K zmiennych.


In [24]:
clf = Pipeline([
    ('select', SelectKBest(k=33)),
    ('model', LogisticRegression(random_state=42, class_weight='balanced'))])

clf.fit(X_train, y_train)
show_scores(clf, X_train, y_train)
show_scores(clf, x_eval, y_eval)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         15821 |          7739 |
|           231 |           709 |

accuracy:              0.6747
precision:             0.0839
recall:                0.7543
f1:                    0.151
roc_auc_discrete:      0.7129
roc_auc_continuous:    0.7852
|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|          9603 |          4796 |
|           162 |           439 |

accuracy:              0.6695
precision:             0.0839
recall:                0.7304
f1:                    0.1504
roc_auc_discrete:      0.6987
roc_auc_continuous:    0.7713


### Prównanie ze zbiorem testowym:

accuracy:              0.687

precision:             0.0941

recall:                0.7311

f1:                    0.1668

roc_auc_discrete:      0.708

roc_auc_continuous:    0.7796

### 3.3.4 Dobór zmiennych na podstawie modelu

Dotychczasowe starania miały na celu osiągnięcie jak najlepszych rezultatów dla modelu. Teraz przeprowadzimy dobór cech na podstawie modelu ze znalezionymi najlepszymi parametrami. Zastosujemy metody L1-based feature selection, Sequential Feature Selection i RFE

### SelectFromModel - L1-based feature selection
Pozmiżej zastosowane parametry mają wartości wyselekcjonowane podczas Grid Searchu | 'C': 0.11288378916846883, 'penalty': 'l1'

In [25]:
clf = LogisticRegression(random_state=42, solver='liblinear', penalty='l1', C=0.11288378916846883, class_weight='balanced')
clf.fit(X_train, y_train)

show_scores(clf, X_train, y_train)
show_scores(clf, x_eval, y_eval)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         16357 |          7203 |
|           213 |           727 |

accuracy:              0.6973
precision:             0.0917
recall:                0.7734
f1:                    0.1639
roc_auc_discrete:      0.7338
roc_auc_continuous:    0.8121
|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|          9918 |          4481 |
|           169 |           432 |

accuracy:              0.69
precision:             0.0879
recall:                0.7188
f1:                    0.1567
roc_auc_discrete:      0.7038
roc_auc_continuous:    0.7893


### Prównanie ze zbiorem testowym:

accuracy:              0.6879

precision:             0.0944

recall:                0.7311

f1:                    0.1672

roc_auc_discrete:      0.7085

roc_auc_continuous:    0.782

In [26]:
X_train.shape[1]

406

In [27]:
sfl = SelectFromModel(clf, prefit=True)
X_train_t = sfl.transform(X_train)
X_eval_t = sfl.transform(x_eval)
X_train_t.shape[1]

114

In [28]:
clf.fit(X_train_t, y_train)

show_scores(clf, X_train_t, y_train)
show_scores(clf, X_eval_t, y_eval)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         16357 |          7203 |
|           213 |           727 |

accuracy:              0.6973
precision:             0.0917
recall:                0.7734
f1:                    0.1639
roc_auc_discrete:      0.7338
roc_auc_continuous:    0.8121
|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|          9918 |          4481 |
|           169 |           432 |

accuracy:              0.69
precision:             0.0879
recall:                0.7188
f1:                    0.1567
roc_auc_discrete:      0.7038
roc_auc_continuous:    0.7893


### Prównanie ze zbiorem testowym:

accuracy:              0.6879

precision:             0.0944

recall:                0.7311

f1:                    0.1672

roc_auc_discrete:      0.7085

roc_auc_continuous:    0.7823

Nie obserwujemy żadnego spadku w wynikach - zreudkowanoy natomiast liczbę cech do 114.

### Podsumowanie

Model regresji logistycznej działa przyzwoicie jak na warunki otrzymanego zbioru danych. Istotne wydaje się użycie parametry class_weight = 'balanced'. 

# 4. Drzewa decyzyjne

## 4.1 Preprocessing

In [29]:
#Kod zespołu budowy
# zarówno dla zmiennych dyskretnych i ciągłych stosujemy nasz transformator ColumnRemover z różnymi parametrami - w kolejnych krokach będziemy szukać najlepszej ich kombinacji

# dokonujemy kodowania one hot encoding zmiennych dyskretnych - traktujemy je jako kategoryczne
int_transformer = Pipeline([
    ('int', ColumnRemover(0.9995, 0.99, 1)),
    ('one_hot', OneHotEncoder(handle_unknown='ignore', sparse_output=False, dtype='int64'))])

# jesteśmy przy drzewach decyzyjnych, więc nie musimy skalować cech
float_transformer = Pipeline([
    ('float', ColumnRemover(0.9999, 0.99, 10))])

col_transformer = ColumnTransformer([
    ('int_pipe', int_transformer, make_column_selector(dtype_include=np.int64)),
    ('float_pipe', float_transformer, make_column_selector(dtype_include=np.float64))
])

## 4.2 Trening pierwszego modelu

In [30]:
X_train = train_df.drop('target', axis=1)
y_train = train_df.target
x_eval = eval_set.drop('target', axis=1)
y_eval = eval_set.target

In [31]:
clf = Pipeline([
    ('preprocessing', col_transformer),
    ('model', DecisionTreeClassifier(random_state=42))])

clf.fit(X_train, y_train)
show_scores(clf, X_train, y_train)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         23560 |             0 |
|             3 |           937 |

accuracy:              0.9999
precision:             1.0
recall:                0.9968
f1:                    0.9984
roc_auc_discrete:      0.9984
roc_auc_continuous:    1.0


In [32]:
show_scores(clf, x_eval, y_eval)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         13807 |           592 |
|           503 |            98 |

accuracy:              0.927
precision:             0.142
recall:                0.1631
f1:                    0.1518
roc_auc_discrete:      0.561
roc_auc_continuous:    0.5609


### Prównanie ze zbiorem testowym:

ccuracy:              0.9257

precision:             0.1429

recall:                0.1467

f1:                    0.1447

roc_auc_discrete:      0.5536

roc_auc_continuous:    0.5536

Model jest zdecydowanie przeuczony. Spróbujmy z parametrem class_weight='balanced'.

In [33]:
clf = Pipeline([
    ('preprocessing', col_transformer),
    ('model', DecisionTreeClassifier(random_state=42, class_weight='balanced'))])

clf.fit(X_train, y_train)
show_scores(clf, X_train, y_train)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         23557 |             3 |
|             0 |           940 |

accuracy:              0.9999
precision:             0.9968
recall:                1.0
f1:                    0.9984
roc_auc_discrete:      0.9999
roc_auc_continuous:    1.0


In [34]:
show_scores(clf, x_eval, y_eval)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         13855 |           544 |
|           529 |            72 |

accuracy:              0.9285
precision:             0.1169
recall:                0.1198
f1:                    0.1183
roc_auc_discrete:      0.541
roc_auc_continuous:    0.541


### Prównanie ze zbiorem testowym:

accuracy:              0.9291

precision:             0.1288

recall:                0.1133

f1:                    0.1206

roc_auc_discrete:      0.5395

roc_auc_continuous:    0.5395

Za dużo to nie dało

## 4.3 Strojenie hiperparametrów i wybór zmiennych

### 4.3.1 Dobór parametrów dla ColumnRemover'a


In [74]:
#Parametry dla narzędzia autorstwa zespołu budowy 
#col_remove_search.best_score_ = 0.5594
            
# col_remove_search.best_params_
#  {'preprocessing__int_pipe__int__threshold_corr': 0.99, 'preprocessing__int_pipe__int__threshold_constant': 0.9998,
#   'preprocessing__int_pipe__int__n_info_vals': 2, 'preprocessing__float_pipe__float__threshold_corr': 0.96,
#   'preprocessing__float_pipe__float__threshold_constant': 0.9998, 'preprocessing__float_pipe__float__n_info_vals': 15}

0.5594 {'preprocessing__int_pipe__int__threshold_corr': 0.99, 'preprocessing__int_pipe__int__threshold_constant': 0.9998, 'preprocessing__int_pipe__int__n_info_vals': 2, 'preprocessing__float_pipe__float__threshold_corr': 0.96, 'preprocessing__float_pipe__float__threshold_constant': 0.9998, 'preprocessing__float_pipe__float__n_info_vals': 15}


### 4.3.2 Regularyzacja modelu

Zajmiemy się regularyzacją modelu drzewa decyzyjnego - będziemy sprawdzać parametry max_depth, min_samples_split, min_samples_leaf, max_features. Na początek jednak skupimy się tylko na max_depth - być może z jego powodu drzewo jest tak mocno przeuczone. Zastosujemy również najlepsze parametry ColumnRemovera znalezione w poprzednim podpunkcie


In [35]:
int_transformer = Pipeline([
    ('int', ColumnRemover(0.9998, 0.99, 2)),
    ('one_hot', OneHotEncoder(handle_unknown='ignore', sparse_output=False, dtype='int64'))])

float_transformer = Pipeline([
    ('float', ColumnRemover(0.9998, 0.96, 15))])

col_transformer = ColumnTransformer([
    ('int_pipe', int_transformer, make_column_selector(dtype_include=np.int64)),
    ('float_pipe', float_transformer, make_column_selector(dtype_include=np.float64))
])

# od tej pory dane testowe i treningowe będą już wstępnie przeprocesowane
X_train = col_transformer.fit_transform(train_df.drop('target', axis=1), train_df.target)
y_train = train_df.target
x_eval = col_transformer.transform(eval_set.drop('target', axis=1))
y_eval = eval_set.target

In [29]:
#Wynik GridSearchu z metryką roc_auc
# depth_search.best_params_={'max_depth': 4}

In [36]:
clf = DecisionTreeClassifier(random_state=42, max_depth=4)
clf = clf.fit(X_train, y_train)
show_scores(clf, X_train, y_train)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         23557 |             3 |
|           925 |            15 |

accuracy:              0.9621
precision:             0.8333
recall:                0.016
f1:                    0.0313
roc_auc_discrete:      0.5079
roc_auc_continuous:    0.8135


In [37]:
show_scores(clf, x_eval, y_eval)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         14387 |            12 |
|           601 |             0 |

accuracy:              0.9591
precision:             0.0
recall:                0.0
f1:                    0.0
roc_auc_discrete:      0.4996
roc_auc_continuous:    0.8066


### Prównanie ze zbiorem testowym:

accuracy:              0.9562

precision:             0.0

recall:                0.0

f1:                    0.0

roc_auc_discrete:      0.4995

roc_auc_continuous:    0.7994

Dość podejrzane te wyniki. Sprawdźmy z metrykę f1 w GridSearchu

In [30]:
#wynik z f1
#depth_search.best_params_={'max_depth': 29}

In [38]:
clf = DecisionTreeClassifier(random_state=42, max_depth=29)
clf = clf.fit(X_train, y_train)
show_scores(clf, X_train, y_train)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         23546 |            14 |
|            29 |           911 |

accuracy:              0.9982
precision:             0.9849
recall:                0.9691
f1:                    0.9769
roc_auc_discrete:      0.9843
roc_auc_continuous:    0.9999


In [39]:
show_scores(clf, x_eval, y_eval)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         13804 |           595 |
|           498 |           103 |

accuracy:              0.9271
precision:             0.1476
recall:                0.1714
f1:                    0.1586
roc_auc_discrete:      0.565
roc_auc_continuous:    0.5699


### Prównanie ze zbiorem testowym:

accuracy:              0.925

precision:             0.1497

recall:                0.16

f1:                    0.1547

roc_auc_discrete:      0.5597

roc_auc_continuous:    0.5665

Sprawdzamy najlepszą kombinację f1

In [86]:
#rand_search.best_params_={'min_samples_split': 3, 'min_samples_leaf': 3, 'max_features': 125, 'max_depth': 25}

0.1182 {'min_samples_split': 3, 'min_samples_leaf': 3, 'max_features': 125, 'max_depth': 25}


In [40]:
clf = DecisionTreeClassifier(random_state=42, max_depth=25, min_samples_split=3, min_samples_leaf=3, max_features=125)
clf.fit(X_train, y_train)
show_scores(clf, X_train, y_train)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         23394 |           166 |
|           439 |           501 |

accuracy:              0.9753
precision:             0.7511
recall:                0.533
f1:                    0.6235
roc_auc_discrete:      0.763
roc_auc_continuous:    0.9878


In [41]:
show_scores(clf, x_eval, y_eval)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         14041 |           358 |
|           524 |            77 |

accuracy:              0.9412
precision:             0.177
recall:                0.1281
f1:                    0.1486
roc_auc_discrete:      0.5516
roc_auc_continuous:    0.6043


### Prównanie ze zbiorem testowym:

accuracy:              0.936

precision:             0.1553

recall:                0.1111

f1:                    0.1295

roc_auc_discrete:      0.542

roc_auc_continuous:    0.5879

Stosowane zabiegi na drzewach nie przyniosły zancznej poprawy

### 4.3.3 Dobór zmiennych nie wymagajacy modelu



### SelectKBest

metryka f1

In [43]:
clf = Pipeline([
    ('select', SelectKBest(k=106)),
    ('model', DecisionTreeClassifier(random_state=42))])

clf.fit(X_train, y_train)
show_scores(clf, X_train, y_train)
show_scores(clf, x_eval, y_eval)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         23560 |             0 |
|            15 |           925 |

accuracy:              0.9994
precision:             1.0
recall:                0.984
f1:                    0.992
roc_auc_discrete:      0.992
roc_auc_continuous:    1.0
|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         13761 |           638 |
|           516 |            85 |

accuracy:              0.9231
precision:             0.1176
recall:                0.1414
f1:                    0.1284
roc_auc_discrete:      0.5486
roc_auc_continuous:    0.5481


### Prównanie ze zbiorem testowym:

accuracy:              0.9239

precision:             0.1531

recall:                0.1711

f1:                    0.1616

roc_auc_discrete:      0.5644

roc_auc_continuous:    0.565

### 4.3.4 Dobór zmiennych na podstawie modelu

Przeprowadzimy dobór cech na podstawie modelu ze znalezionymi najlepszymi parametrami. Zastosujemy metody SelectFromModel - feature importance, Sequential Feature Selection i RFE

### SelectFromModel - tree-based feature selection

In [44]:
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

show_scores(clf, X_train, y_train)
show_scores(clf, x_eval, y_eval)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         23560 |             0 |
|             3 |           937 |

accuracy:              0.9999
precision:             1.0
recall:                0.9968
f1:                    0.9984
roc_auc_discrete:      0.9984
roc_auc_continuous:    1.0
|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         13805 |           594 |
|           497 |           104 |

accuracy:              0.9273
precision:             0.149
recall:                0.173
f1:                    0.1601
roc_auc_discrete:      0.5659
roc_auc_continuous:    0.5658


### Prównanie ze zbiorem testowym:

accuracy:              0.9261

precision:             0.1517

recall:                0.1578

f1:                    0.1547

roc_auc_discrete:      0.5591

roc_auc_continuous:    0.5591

In [45]:
X_train.shape[1]

197

In [46]:
sfl = SelectFromModel(clf, prefit=True)
X_train_t = sfl.transform(X_train)
X_eval_t = sfl.transform(x_eval)
X_train_t.shape[1]

22

In [47]:
clf.fit(X_train_t, y_train)

show_scores(clf, X_train_t, y_train)
show_scores(clf, X_eval_t, y_eval)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         23560 |             0 |
|             3 |           937 |

accuracy:              0.9999
precision:             1.0
recall:                0.9968
f1:                    0.9984
roc_auc_discrete:      0.9984
roc_auc_continuous:    1.0
|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         13771 |           628 |
|           489 |           112 |

accuracy:              0.9255
precision:             0.1514
recall:                0.1864
f1:                    0.167
roc_auc_discrete:      0.5714
roc_auc_continuous:    0.5712


### Prównanie ze zbiorem testowym:

accuracy:              0.9211

precision:             0.1337

recall:                0.1533

f1:                    0.1429

roc_auc_discrete:      0.5544

roc_auc_continuous:    0.5544

Powyższy wynik jest przy 22 cechach, natomiast wyniki metryk są pogorszone.

### 4.4 Podsumowanie

Model drzewa decyzyjnego działa bardzo słabo w przypadku tego zbioru danych. Ma tendencje do overfittingu. Nie jesteśmy w stanie nic z tym zrobić. 

Natomiast redukcja zmiennych:
- SelectKBest - 106
- Tree-based feature selection (feature_importance) - 22

Zespół budowy słusznie zauważa duże znaczenie parametru feature importance.



# 5. SVM

## 5.1 Preprocessing

In [31]:
int_transformer = Pipeline([
    ('int', ColumnRemover(0.9995, 0.99, 1)),
    ('one_hot', OneHotEncoder(handle_unknown='ignore', sparse_output=False, dtype='int64'))])

float_transformer = Pipeline([
    ('float', ColumnRemover(0.9999, 0.99, 10)),
    ('standard_scaler', StandardScaler())])

col_transformer = ColumnTransformer([
    ('int_pipe', int_transformer, make_column_selector(dtype_include=np.int64)),
    ('float_pipe', float_transformer, make_column_selector(dtype_include=np.float64))
])

In [32]:
X_train = train_df.drop('target', axis=1)
y_train = train_df.target
x_eval = eval_set.drop('target', axis=1)
y_eval = eval_set.target

## 5.2 Trening pierwszego modelu

In [50]:
clf = Pipeline([
    ('preprocessing', col_transformer),
    ('model', SVC(random_state=42, probability=True))])

clf.fit(X_train, y_train)
show_scores(clf, X_train, y_train)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         23560 |             0 |
|           926 |            14 |

accuracy:              0.9622
precision:             1.0
recall:                0.0149
f1:                    0.0294
roc_auc_discrete:      0.5074
roc_auc_continuous:    0.8062


In [51]:
#wyniki dla zbioru walidacyjnego
show_scores(clf, x_eval, y_eval)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         14399 |             0 |
|           601 |             0 |

accuracy:              0.9599
precision:             0.0
recall:                0.0
f1:                    0.0
roc_auc_discrete:      0.5
roc_auc_continuous:    0.6425


### Prównanie ze zbiorem testowym:

accuracy:              0.9571

precision:             0.0

recall:                0.0

f1:                    0.0

roc_auc_discrete:      0.5

roc_auc_continuous:    0.494


### Podsumowująć Support Vector Machine się nie nadaje 

# 6. Random forest

## 6.1 Preprocessing

In [52]:
int_transformer = Pipeline([
    ('int', ColumnRemover(0.9995, 0.99, 1)),
    ('one_hot', OneHotEncoder(handle_unknown='ignore', sparse_output=False, dtype='int64'))])

float_transformer = Pipeline([
    ('float', ColumnRemover(0.9999, 0.99, 10))])

col_transformer = ColumnTransformer([
    ('int_pipe', int_transformer, make_column_selector(dtype_include=np.int64)),
    ('float_pipe', float_transformer, make_column_selector(dtype_include=np.float64))
])

## 6.2 Trening modelu

In [34]:
X_train = train_df.drop('target', axis=1)
y_train = train_df.target
x_eval = eval_set.drop('target', axis=1)
y_eval = eval_set.target

In [35]:
rf = RandomForestClassifier(random_state=42, n_estimators=1000)

clf = Pipeline([
    ('preprocessing', col_transformer),
    ('model', rf)])

clf.fit(X_train, y_train)
show_scores(clf, X_train, y_train)
show_scores(clf, x_eval, y_eval)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         23560 |             0 |
|             3 |           937 |

accuracy:              0.9999
precision:             1.0
recall:                0.9968
f1:                    0.9984
roc_auc_discrete:      0.9984
roc_auc_continuous:    1.0
|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         14369 |            30 |
|           589 |            12 |

accuracy:              0.9587
precision:             0.2857
recall:                0.02
f1:                    0.0373
roc_auc_discrete:      0.5089
roc_auc_continuous:    0.8047


### Prównanie ze zbiorem testowym:

accuracy:              0.9557

precision:             0.2414

recall:                0.0156

f1:                    0.0292

roc_auc_discrete:      0.5067

roc_auc_continuous:    0.7948

### Zespół przeczucza las losowy

# Próba z parametrami

In [55]:
rf = RandomForestClassifier(random_state=42, n_estimators=1000, min_samples_split=3, min_samples_leaf=3, max_features=125, max_depth=25)

clf = Pipeline([
    ('preprocessing', col_transformer),
    ('model', rf)])

clf.fit(X_train, y_train)
show_scores(clf, X_train, y_train)
show_scores(clf, x_eval, y_eval)


|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         23560 |             0 |
|           784 |           156 |

accuracy:              0.968
precision:             1.0
recall:                0.166
f1:                    0.2847
roc_auc_discrete:      0.583
roc_auc_continuous:    0.9986
|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         14396 |             3 |
|           601 |             0 |

accuracy:              0.9597
precision:             0.0
recall:                0.0
f1:                    0.0
roc_auc_discrete:      0.4999
roc_auc_continuous:    0.8239


### Prównanie ze zbiorem testowym:

accuracy:              0.9571

precision:             0.0

recall:                0.0

f1:                    0.0

roc_auc_discrete:      0.5

roc_auc_continuous:    0.8218

### Wniosek: las losowy również się nie nadaje

# 7. Voting


Zespół budowy słusznie wybiera najlepsze modele, które wezmą udział w votingu.

In [74]:
log_reg_vc = Pipeline([
    ('preprocessor', ColumnTransformer([
        ('int_pipe', Pipeline([
            ('int', ColumnRemover(0.9998, 1, 0)),
            ('one_hot', OneHotEncoder(handle_unknown='ignore', sparse_output=False, dtype='int64'))]), make_column_selector(dtype_include=np.int64)),
        ('float_pipe', Pipeline([
            ('float', ColumnRemover(0.9996, 0.97, 0)),
            ('standardization', StandardScaler())]), make_column_selector(dtype_include=np.float64))
    ])),
    ('selector', SelectKBest(k=33)),
    ('model', LogisticRegression(random_state=42, class_weight='balanced'))])

dct_vc = Pipeline([
    ('preprocessor', ColumnTransformer([
        ('int_pipe', Pipeline([
            ('int', ColumnRemover(0.9998, 0.99, 2)),
            ('one_hot', OneHotEncoder(handle_unknown='ignore', sparse_output=False, dtype='int64'))]), make_column_selector(dtype_include=np.int64)),
        ('float_pipe', Pipeline([
            ('float', ColumnRemover(0.9998, 0.96, 15)),
            ]), make_column_selector(dtype_include=np.float64))
    ])),
    ('selector', SelectFromModel(DecisionTreeClassifier(random_state=42))),
    ('model', DecisionTreeClassifier(random_state=42))])

svc_vc = Pipeline([
    ('preprocessor', ColumnTransformer([
        ('int_pipe', Pipeline([
            ('int', ColumnRemover(0.9995, 0.99, 1)),
            ('one_hot', OneHotEncoder(handle_unknown='ignore', sparse_output=False, dtype='int64'))]), make_column_selector(dtype_include=np.int64)),
        ('float_pipe', Pipeline([
            ('float', ColumnRemover(0.9999, 0.99, 10)),
            ('standard_scaler', StandardScaler())]), make_column_selector(dtype_include=np.float64))
    ])),
    ('model', SVC(random_state=42, probability=True))])

In [61]:
estimators=[('DecisionTree', dct_vc), ('SVM', svc_vc), ('LR', log_reg_vc)]
vc = VotingClassifier(estimators=estimators, voting='soft', weights=[0.2, 0.1, 0.7]).fit(X_train, y_train)


AttributeError: 'numpy.int64' object has no attribute 'index'

# Powyższa funkcja zwraca AttributeError : 'numpy.int64' object has no attribute 'index'
### Problem powstaje przy fitowaniu danych
 notebook wykonywany jest poprawnie jednak przez wiele godzin nie można było znaleść przyczyny błędy
 sprawdzaliśmy wymiarowość oraz typ danych wchodzących do metody .fit()

In [62]:
#walidacja: sprawdzam error 
eestimators = [('clf1', dct_vc), ('clf2', svc_vc), ('clf3', log_reg_vc)]

for name, estimator in eestimators:
    print(name, type(estimator))

clf1 <class 'sklearn.pipeline.Pipeline'>
clf2 <class 'sklearn.pipeline.Pipeline'>
clf3 <class 'sklearn.pipeline.Pipeline'>


In [63]:

#waliadacja: sprawdzam error
if isinstance(X_train, (pd.DataFrame)) and X_train.ndim == 2:
    print('X_train is a 2D data frame')
else:
    print('X_train is not a 2D data frame')

if isinstance(y_train, (pd.Series)) and y_train.ndim == 1:
    print('y_train is a 1D Series')
else:
    print('y_train is not a 1D Series')

X_train is a 2D data frame
y_train is a 1D Series


In [64]:
#walidacja: sprawdzam error
print(y_train.dtypes)

int64


 Przykłady rozwiązań tego problemu, które udało się znaleść przekopując internet dotyczyły niespełnienia któregoś z powyższych warunków, które zostały wyprintowane.

 Chat gpt również zawiódł, sugerował że problem z typem danych i nie powiedział nic czego byśmy nie sprawdzili żeby to później się zapętlić i wkoło opowiadać te same rozwiązania (które nie działały bo tylko na tym został wytrenowany).




### Rozwiązanie problemu:
 Okazało się, że AttributeError pojawiający się przy .fit() wynika z błędnego napisania klasy 'ColumnRemover'.
 Aby poprawnie przeprowadzić voting należałoby zakomentować część tej klasy i dopiero później odpalić voting.
 Zespół budowy nie umieścił żadnej informacji na ten temat, a wystarczyłby komentarz nad kodem.



### Wnioski:
 Winowającą okazała się część funkcji z klasy ColumnRemover, która była odpowiedzialna za usunięcie kolum nie niosących informacji.
 Pisanie własnych, skomplikowanych funkcji przetwarzących dane, a szczególnie takie nierzeczywiste jest ciężkim zadaniem. Prościej jest korzystać z gotowych rozwiązań. Obniża to możliwość pojawienia się nieprawidłowości.
 Ważne jest również komentowanie kodu.

In [75]:
estimators=[('DecisionTree', dct_vc), ('SVM', svc_vc), ('LR', log_reg_vc)]
vc = VotingClassifier(estimators=estimators, voting='soft', weights=[0.2, 0.1, 0.7]).fit(X_train, y_train)

In [76]:
show_scores(vc, X_train, y_train)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         21395 |          2165 |
|           179 |           761 |

accuracy:              0.9043
precision:             0.2601
recall:                0.8096
f1:                    0.3937
roc_auc_discrete:      0.8588
roc_auc_continuous:    0.9449


In [77]:
show_scores(vc, x_eval, y_eval)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         12838 |          1561 |
|           334 |           267 |

accuracy:              0.8737
precision:             0.1461
recall:                0.4443
f1:                    0.2198
roc_auc_discrete:      0.6679
roc_auc_continuous:    0.7784


### Prównanie ze zbiorem testowym:

accuracy:              0.8722

precision:             0.1621

recall:                0.4756

f1:                    0.2418

roc_auc_discrete:      0.6828

roc_auc_continuous:    0.7792



### Niespotykanie wysoki jak dotąd rezultat f1 oraz całkiem przyzwoite inne metryki.

# 8. Adaboost

### Regresja Logistyczna

In [21]:
log_reg = Pipeline([
    ('preprocessor', ColumnTransformer([
        ('int_pipe', Pipeline([
            ('int', ColumnRemover(0.9998, 1, 0)),
            ('one_hot', OneHotEncoder(handle_unknown='ignore', sparse_output=False, dtype='int64'))]), make_column_selector(dtype_include=np.int64)),
        ('float_pipe', Pipeline([
            ('float', ColumnRemover(0.9996, 0.97, 0)),
            ('standardization', StandardScaler())]), make_column_selector(dtype_include=np.float64))
    ])),
    ('selector', SelectKBest(k=33)),
    ('model', LogisticRegression(random_state=42, class_weight='balanced'))])

In [22]:
ADB_logreg = AdaBoostClassifier(base_estimator=log_reg, n_estimators=5000,
    learning_rate=0.5, random_state=42)

In [23]:
ADB_logreg.fit(X_train, y_train)
show_scores(ADB_logreg, X_train,y_train)

ValueError: NoneType doesn't support sample_weight.

In [24]:
show_scores(ADB_logreg, x_eval,y_eval)

AttributeError: 'AdaBoostClassifier' object has no attribute 'n_classes_'

### Drzewa decyzyjne

In [82]:
dct = Pipeline([
    ('preprocessor', ColumnTransformer([
        ('int_pipe', Pipeline([
            ('int', ColumnRemover(0.9998, 0.99, 2)),
            ('one_hot', OneHotEncoder(handle_unknown='ignore', sparse_output=False, dtype='int64'))]), make_column_selector(dtype_include=np.int64)),
        ('float_pipe', Pipeline([
            ('float', ColumnRemover(0.9998, 0.96, 15)),
        ]), make_column_selector(dtype_include=np.float64))
    ])),
    ('selector', SelectFromModel(DecisionTreeClassifier(random_state=42))),
    ('model', DecisionTreeClassifier(random_state=42))])

In [83]:
ADB_dct = AdaBoostClassifier(base_estimator=dct, n_estimators=1000,learning_rate=0.5,
                         algorithm='SAMME.R', random_state=42)

In [84]:
ADB_dct.fit(X_train, y_train)
show_scores(ADB_dct, X_train,y_train)
show_scores(ADB_dct, x_eval,y_eval)

ValueError: NoneType doesn't support sample_weight.

In [85]:
ADB_dct = AdaBoostClassifier(base_estimator=dct, n_estimators=2000,learning_rate=0.5,
                         algorithm='SAMME.R', random_state=42)

In [86]:
ADB_dct.fit(X_train, y_train)
show_scores(ADB_dct, X_train,y_train)
show_scores(ADB_dct, x_eval,y_eval)

ValueError: NoneType doesn't support sample_weight.

### Kolejny error: tym razem AdaBoost

Po wielu godzinach ponownego i intensywnego researchu w celu rozwiązania problemu nie udało się rozwiązać kwestii tego erroru. Kod w repozytorium zaespołu budowy na githubie jest wykonany i ma outputy. 
Natomiast w celu weryfikacji wyników notebooka budowy i sprawdzenia czy oryginał zadziała na zbiorze treningowym i testowym postanowiliśmy wykonać go raz jeszcze.
Po odpaleniu oryginalnego notebooka również pojawia się ten sam błąd.

### Wniosek:
### Kod grupy budowy nie wykonuje się poprawnie.

# Próba rozwiązania

### Spróbowaliśmy rozwiązać kwestię pojawienia się komunikatu - ValueError: NoneType doesn't support sample_weight.
Dodaliśmy sample_weight=None.
Kod nie działa zarówno w wersji zespołu budowy z repozytorium na danych treningowych i testowych jak i na danych treningowych i walidacyjnych. 

In [27]:
ADB_logreg.fit(X_train, y_train, sample_weight=None)
show_scores(ADB_logreg, X_train,y_train)

ValueError: NoneType doesn't support sample_weight.

### Próby rozwiązania do niczego nie doprowadziły

# Przejdźmy do podsumowania

### Wyniki roc_auc na danych testowych

Regresja logistyczna bez param.: 0.781 |
Regresja logistyczna z param.: 0.782
| Ilości zmiennych przy niepogorszonych wynikach
w regresji logistycznej:
|-K_best: 33
|-SFM: 114
|-SFS: 40
|-RFE: 72
|-PCA: 3 |

Drzewo decyzyjne bez param.: 0.554 |
Drzewo decyzyjne z param.: 0.588 
| Ilości zmiennych przy niepogorszonych wynikach
w drzewie decyzyjnym:
|-K_best: 106
|-SFM: 22
|-SFS: 70
|-RFE: 26
|-PCA: 14 |

|Model   |No_hyper   |with_hyper   |
|---|---|---|
|SVC   |0.670   |0.768   |
|KNN   |0.542 |0.570   |
|rand_forest   |0.794   | 0.821   |
|AdaBoost logreg  |0.774   |0.799   |
|AdaBoost dct  |0.676   |0.684   |
|Voting   | ***  | 0.779  | 

# Walidacja - dane walidacyjne
 ? - brak możliwości weryfikacji wyniku |
 @ - brak implementacji w ostatecznym projekcie
### Wyniki roc_auc na danych walidacyjnych

Regresja logistyczna bez param.: 0.786 |
Regresja logistyczna z param.: 0.789
| Ilości zmiennych przy niepogorszonych wynikach
w regresji logistycznej:
|-K_best: 33
|-SFM: 114
|-SFS: ?
|-RFE: ?
|-PCA: ? |

Drzewo decyzyjne bez param.: 0.561 |
Drzewo decyzyjne z param.: 0.604 
| Ilości zmiennych przy niepogorszonych wynikach
w drzewie decyzyjnym:
|-K_best: 33
|-SFM: 114
|-SFS: ?
|-RFE: ?
|-PCA: ? | 


|Model   |No_hyper   |with_hyper   |
|---|---|---|
|SVC   |0.642  |  ?     |
|KNN   |  @   |  @     |
|rand_forest   |0.806   | 0.824   |
|AdaBoost logreg  |  ?     |    ?   |
|AdaBoost dct  |  ?     |  ?     |
|Voting   | ***  | 0.778  | 


# Podsumowanie całej walidacji 
Zespół budowy nie miał łatwego zadania. Ich dataset jest nieprzyjazny, a potęguje to fakt syntetyczności danych nie reprezentujących dosłownie niczego. 


### Pozytywy:
Zespół budowy wykazał się pomysłowością i kreatywnością. Zrobił wyczerującą i obszerną analizę eksploracyjną. Stworzył również do tego własne autorskie narzędzia. 

Zespół wykazał się doborem  modeli, stosowaniem rozmaitych technik inżynierii cech wraz z parametrami oraz selekcji hiperparametrów dla modeli. Przeszukał ogromne przestrzenie parametrów w celu otrzymania jak najlepszego wyniku w zależności od modelu. Powyższe kosztowało ich dużo cierpliwości i zużytego prądu.

Poprawnie interpretował metryki oraz wnioskował na ich podstawie w celu poprawienia wyników modeli.


### Negatywy:
Tak jak zauważamy na samym początku walidacji brak pliku z wersjami paczek - requirements. Bez podania wersji i cech środowiska programistycznego nie można odtworzyć wyników badania. Sprawia to, że w świecie naukowym czy w pewnych przypadkach komercyjnych projekt jest zwyczajnie do kosza.

Cały projekt jest w 1 notebooku. Dobrą praktyką jest rozdzielenie projektu na różne pliki w tym na oddzielne skrypty, której mają za zadanie przechowywać narzędzia wywoływane w projekcie. Ma to oszczędzić miejsce i sprawia, że projekt jest czytelniejszy.

Stosowanie stworzonych w całości przez autorów narzędzi (w tym przypadku ColumnRemovera) i niepoinformowanie czy brak budowy działającej funkcji spowodował niemożność wykonania operacji votingu. Zespół walidacji niepotrzebnie stracił wiele godzin na próbę rozwiązania problemu. Proponujemy stosowanie gotowych rozwiązań z istniejących bibliotek.

Stosowanie PCA przez autorów do drastycznej redukcji wymiarowości zmiennych. Nie zawsze chcemy dokonywać przkształceń liniowych, które mogą zaburzyć nam predykcyjność/poprawę predykcyjności modelu. Tzn. redukcja ilości zmiennych do zmiennych nieskorelowanych może mieć zły wpływ na działanie modelu. Również zaburza ważość cech co negatywnie wpływa na wyjaśnialność modelu. Wyjaśnialność modelu jest kluczowa w zadaniach takich jak klasyfikacja.  Bibliografia (https://www.yourdatateacher.com/2022/07/11/why-you-shouldnt-use-pca-in-a-supervised-machine-learning-project/), (https://blog.kxy.ai/5-reasons-you-should-never-use-pca-for-feature-selection/), (https://www.kaggle.com/general/215905),

Wyniki modeli: Zespół budowy w powyższej tabli z podsumowaniem metryki roc_auc i ilości cech, zawierającej wyniki budowy (z danych treningowych i testowych ) podaje wyniki z niewiadomo skąd, w głównym notebooku brak outputu o ilości cech PCA i RFE oraz metrykach support vector machine z dobranymi parametrami. Domyślamy się iż jest to spowodowane długim czasem oczekiwania i dużą złożonością obliczeniową (nam też nie chciało się obliczyć). Jednakże w takim przypadku należałoby nie umieszczać wyników, których nie możemy odtworzyć i zobaczyć, nie wierzyć na słowo autorom.

Powyższa uwaga tyczy się umieszczenia modelu K-nearest neighbouhrs, tego modelu zwyczajnie nie ma w finalnym pliku.

Ostatnią uwagą jest brak wyboru modelu wdrożeniowego. Mimo kompletnego braku kontekstu danych i ich bezsensowności zwasze należy wybrać model, ponieważ to jest sedno projektu. Tabelka z wynikami metryk modeli nic nie mówi. Projekt zwyczajnie nie jest skończony. 






